In [ ]:
import os

# Manually set the environment variable
os.environ["GITHUB_TOKEN"] = "*Place your token here to access API*"

# Confirm it was set
print("GITHUB_TOKEN:", os.getenv('GITHUB_TOKEN'))
headers = {
    "Authorization": f"token {os.getenv('GITHUB_TOKEN')}",
    "Accept": "application/vnd.github.v3+json"
}


This is the updated code for the users from delhi with large followers

In [ ]:
#This is the code to scrape the users from Delhi with more than 100 followers.
import requests
import json
import time
import os

# Authentication and rate limiting
token = os.getenv("GITHUB_TOKEN")
headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
}
# rate_limit_per_hour = 700  # GitHub API rate limit
# sleep_interval = 3600 / rate_limit_per_hour  # Approximate delay per request

# Parameters for fetching users
location = "Delhi"
min_followers = 100
per_page = 30  # Max items per page in GitHub API
max_user_pages = 25  # Control the number of pages for fetching users

# Base URL for user search and individual user details
search_url = "https://api.github.com/search/users"
user_detail_url_template = "https://api.github.com/users/{}"

# List to store user data
user_data_list = []

for page in range(1, max_user_pages + 1):
    # Search users by location and minimum follower count
    params = {
        "q": f"location:{location} followers:>{min_followers}",
        "per_page": per_page,
        "page": page
    }

    response = requests.get(search_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()

        for user in data["items"]:
            username = user["login"]

            # Fetch individual user details
            user_detail_response = requests.get(user_detail_url_template.format(username), headers=headers)
            if user_detail_response.status_code == 200:
                user_data = user_detail_response.json()

                # Extract the required fields
                user_info = {
                    "login": user_data.get("login"),
                    "name": user_data.get("name"),
                    "company": user_data.get("company"),
                    "location": user_data.get("location"),
                    "email": user_data.get("email"),
                    "hireable": user_data.get("hireable"),
                    "bio": user_data.get("bio"),
                    "public_repos": user_data.get("public_repos"),
                    "followers": user_data.get("followers"),
                    "following": user_data.get("following"),
                    "created_at": user_data.get("created_at")
                }
                print(user_info)

                # Append to the list
                user_data_list.append(user_info)

                # Delay to respect rate limits
                # time.sleep(sleep_interval)
            else:
                print(f"Failed to fetch details for user: {username}")

        # Stop if we get fewer users than `per_page` (indicating last page)
        if len(data["items"]) < per_page:
            break

        # Delay between pages
        # time.sleep(2)
    else:
        print(f"Error: {response.status_code}")
        break

# Save data to JSON file
with open("/content/github_user_delhi.json", "w") as f:
    json.dump(user_data_list, f, indent=4)

print("Data saved to github_user_delhi.json")


In [ ]:
import json
import csv

# File paths
json_file_path = "/content/github_user_delhi.json"
csv_file_path = "/content/github_user_delhi.csv"

# Load JSON data
with open(json_file_path, "r") as json_file:
    data = json.load(json_file)

# Define the CSV field names (headers)
fieldnames = ["login", "name", "company", "location", "email", "hireable", "bio",
              "public_repos", "followers", "following", "created_at"]

# Write to CSV file
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header row

    # Write each user's data row
    for user_info in data:
        writer.writerow(user_info)

print("Data successfully converted to github_users_delhi.csv")


This is the second part of the project which creates repo.csv


In [ ]:
#This code creates the repository.csv by scraping it from Github.
import requests
import csv
import time
import os

# Authentication and rate limiting
token = os.getenv("GITHUB_TOKEN")
headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
}
# rate_limit_per_hour = 700  # GitHub API rate limit
# sleep_interval = 3600 / rate_limit_per_hour  # Approximate delay per request

# Parameters for fetching repositories
per_page = 100  # Max items per page in GitHub API
max_repos = 500  # Max repositories to fetch per user

# Base URL for fetching user repositories
repos_url_template = "https://api.github.com/users/{}/repos"

# File paths
users_csv_path = "/content/github_user_delhi.csv"
repos_csv_path = "/content/repositories.csv"

# Define the CSV field names for repositories.csv
repo_fieldnames = [
    "login", "full_name", "created_at", "stargazers_count",
    "watchers_count", "language", "has_projects", "has_wiki", "license_name"
]

# List to store repository data
repo_data_list = []

# Open users.csv to read usernames and repositories.csv to write repository data
with open(users_csv_path, "r") as users_file:
    users_reader = csv.DictReader(users_file)

    # Process each user
    for user in users_reader:
        username = user["login"]  # The GitHub user ID (login) of the owner
        print(f"Fetching repositories for user: {username}")

        # Fetch repositories for the user
        repo_count = 0
        for page in range(1, (max_repos // per_page) + 2):
            params = {
                "per_page": per_page,
                "page": page,
                "sort": "pushed"  # Sort by most recently pushed
            }

            response = requests.get(repos_url_template.format(username), headers=headers, params=params)
            if response.status_code == 200:
                repos = response.json()

                for repo in repos:
                    repo_info = {
                        "login": repo["owner"]["login"],  # User ID of the owner
                        "full_name": repo.get("full_name"),
                        "created_at": repo.get("created_at"),
                        "stargazers_count": repo.get("stargazers_count"),
                        "watchers_count": repo.get("watchers_count"),
                        "language": repo.get("language"),
                        "has_projects": repo.get("has_projects"),
                        "has_wiki": repo.get("has_wiki"),
                        "license_name": repo["license"]["name"] if repo.get("license") else None
                    }

                    # Add repository info to the list
                    repo_data_list.append(repo_info)

                    repo_count += 1
                    if repo_count >= max_repos:
                        break

                # Break out of the loop if fewer than `per_page` results are returned (end of repos)
                if len(repos) < per_page or repo_count >= max_repos:
                    break

                # Delay to respect rate limits
                # time.sleep(sleep_interval)
            else:
                print(f"Failed to fetch repositories for user: {username}, Status Code: {response.status_code}")
                break

# Save repositories data to CSV
with open(repos_csv_path, mode="w", newline="", encoding="utf-8") as repos_file:
    repos_writer = csv.DictWriter(repos_file, fieldnames=repo_fieldnames)
    repos_writer.writeheader()
    repos_writer.writerows(repo_data_list)

print("Repository data saved to repositories.csv")


This is sorted by created at

In [ ]:
#This sorts the users.csv by created_at column and saves it in new file.
import csv
from datetime import datetime

# File path for users.csv
users_csv_path = "/content/users.csv"
sorted_users_csv_path = "/content/sorted_users.csv"

# Read users from users.csv
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)
    users_list = list(users_reader)  # Convert to a list for sorting

# Sort users by created_at (in ascending order)
sorted_users_list = sorted(users_list, key=lambda x: datetime.strptime(x['created_at'], '%Y-%m-%dT%H:%M:%SZ'))

# Write sorted users to a new CSV file
with open(sorted_users_csv_path, mode="w", newline="", encoding="utf-8") as sorted_users_file:
    fieldnames = users_reader.fieldnames  # Use the same field names
    sorted_users_writer = csv.DictWriter(sorted_users_file, fieldnames=fieldnames)
    sorted_users_writer.writeheader()
    sorted_users_writer.writerows(sorted_users_list)

print(f"Users sorted by 'created_at' and saved to {sorted_users_csv_path}")


In [ ]:
#This finds the top three licenses used.
import csv
from collections import Counter

# File path for repositories.csv
repos_csv_path = "/content/repositories.csv"

# Initialize a counter for licenses
license_counter = Counter()

# Read repositories from repositories.csv
with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    # Count licenses while ignoring missing values
    for repo in repos_reader:
        license_name = repo.get("license_name")
        if license_name:  # Ignore missing licenses
            license_counter[license_name] += 1

# Get the three most common licenses
most_common_licenses = license_counter.most_common(3)

# Display the results
print("Three Most Popular Licenses:")
for license_name, count in most_common_licenses:
    print(f"{license_name}: {count} occurrences")


Cleaning Company Name

In [ ]:
import csv

# File paths for users.csv and cleaned_users.csv
users_csv_path = "/content/users.csv"
cleaned_users_csv_path = "/content/users.csv"

# Read users from users.csv and clean up company names
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    # Prepare a list to hold cleaned user data
    cleaned_users_list = []

    for user in users_reader:
        # Clean the company name
        company = user.get("company", "").strip()  # Get company and trim whitespace
        if company.startswith('@'):
            company = company[1:]  # Strip leading @ symbol
        company = company.upper()  # Convert to uppercase

        # Update the user's company name
        user["company"] = company

        # Add the cleaned user data to the list
        cleaned_users_list.append(user)

# Write the cleaned user data to a new CSV file
with open(cleaned_users_csv_path, mode="w", newline="", encoding="utf-8") as cleaned_users_file:
    fieldnames = users_reader.fieldnames  # Use the same field names
    cleaned_users_writer = csv.DictWriter(cleaned_users_file, fieldnames=fieldnames)
    cleaned_users_writer.writeheader()
    cleaned_users_writer.writerows(cleaned_users_list)

print(f"Cleaned company names saved to {cleaned_users_csv_path}")


In [ ]:
import csv

# File path for users.csv
input_csv_path = "/content/users.csv"
output_csv_path = "/content/users.csv"

# Step 1: Read and process users.csv
processed_users = []

with open(input_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        # Convert hireable values: "True" to "true", "False" to "false", and None or empty strings as "null"
        if user.get("hireable") == "True":
            user["hireable"] = "true"
        elif user.get("hireable") == "False":
            user["hireable"] = "false"
        elif user.get("hireable") in ["", None]:
            user["hireable"] = "null"

        # Append modified user data to processed_users list
        processed_users.append(user)

# Step 2: Write the processed data back to a new CSV file
with open(output_csv_path, mode="w", newline='', encoding="utf-8") as output_file:
    fieldnames = processed_users[0].keys()
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(processed_users)

print(f"Processed data saved to {output_csv_path}")


The users.csv is completely cleaned here above. Now in this step repo.csv is cleaned.

In [ ]:
import csv

# File path for repositories.csv
input_csv_path = "/content/repositories.csv"
output_csv_path = "/content/repositories.csv"

# Step 1: Read and process repositories.csv
processed_repos = []

with open(input_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    for repo in repos_reader:
        # Convert has_projects values: "True" to "true", "False" to "false", and None or empty strings as "null"
        if repo.get("has_projects") == "True":
            repo["has_projects"] = "true"
        elif repo.get("has_projects") == "False":
            repo["has_projects"] = "false"
        elif repo.get("has_projects") in ["", None]:
            repo["has_projects"] = "null"

        # Convert has_wiki values: "True" to "true", "False" to "false", and None or empty strings as "null"
        if repo.get("has_wiki") == "True":
            repo["has_wiki"] = "true"
        elif repo.get("has_wiki") == "False":
            repo["has_wiki"] = "false"
        elif repo.get("has_wiki") in ["", None]:
            repo["has_wiki"] = "null"

        # Append modified repo data to processed_repos list
        processed_repos.append(repo)

# Step 2: Write the processed data back to a new CSV file
with open(output_csv_path, mode="w", newline='', encoding="utf-8") as output_file:
    fieldnames = processed_repos[0].keys()
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(processed_repos)

print(f"Processed data saved to {output_csv_path}")


In [ ]:
#This code finds the company from which maximum developers belong.
import csv
from collections import Counter

# File path for cleaned_users.csv
cleaned_users_csv_path = "/content/users.csv"

# Initialize a counter for companies
company_counter = Counter()

# Read users from cleaned_users.csv
with open(cleaned_users_csv_path, mode="r", encoding="utf-8") as cleaned_users_file:
    users_reader = csv.DictReader(cleaned_users_file)

    # Count occurrences of each company
    for user in users_reader:
        company = user.get("company")
        if company:  # Ignore missing company names
            company_counter[company] += 1

# Find the company with the majority of developers
most_common_company, most_common_count = company_counter.most_common(1)[0]

# Display the result
print(f"The majority of developers work at: {most_common_company} with {most_common_count} developers.")


In [ ]:
#This finds the most used Programming Language by devs.
import csv
from collections import Counter

# File path for repositories.csv
repos_csv_path = "/content/repositories.csv"

# Initialize a counter for programming languages
language_counter = Counter()

# Read repositories from repositories.csv
with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    # Count occurrences of each programming language
    for repo in repos_reader:
        language = repo.get("language")
        if language:  # Ignore missing languages
            language_counter[language] += 1

# Find the most popular programming language
most_common_language, most_common_count = language_counter.most_common(1)[0]

# Display the result
print(f"The most popular programming language is: {most_common_language} with {most_common_count} repositories.")


In [ ]:
#This finds the most used Programming Language by devs who joined after 2020.
import csv
from collections import Counter
from datetime import datetime

# File paths for users.csv and repositories.csv
users_csv_path = "/content/users.csv"
repos_csv_path = "/content/repositories.csv"

# Set the threshold date for filtering users
threshold_date = datetime(2020, 1, 1)

# Step 1: Read users from users.csv and filter based on created_at
new_users = set()  # Set to store the login names of users who joined after 2020

with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        created_at = datetime.strptime(user["created_at"], '%Y-%m-%dT%H:%M:%SZ')
        if created_at > threshold_date:
            new_users.add(user["login"])  # Store the login names of filtered users

# Step 2: Count programming languages for the filtered users in repositories.csv
language_counter = Counter()

with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    for repo in repos_reader:
        if repo["login"] in new_users:  # Check if the repository belongs to a new user
            language = repo.get("language")
            if language:  # Ignore missing languages
                language_counter[language] += 1

# Step 3: Find the second most popular programming language
if len(language_counter) >= 2:
    # Get the two most common languages
    most_common_languages = language_counter.most_common(2)
    second_most_common_language, second_most_common_count = most_common_languages[1]
    print(f"The second most popular programming language among users who joined after 2020 is: "
          f"{second_most_common_language} with {second_most_common_count} repositories.")
else:
    print("Not enough data to determine the second most popular programming language.")


In [ ]:
#This finds the highest average number of stars per repository.
import csv
from collections import defaultdict

# File path for repositories.csv
repos_csv_path = "/content/repositories.csv"

# Initialize dictionaries to hold total stars and counts of repositories per language
language_stars = defaultdict(int)  # Total stars per language
language_counts = defaultdict(int)  # Count of repositories per language

# Step 1: Read repositories from repositories.csv and calculate stars and counts
with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    for repo in repos_reader:
        language = repo.get("language")
        stargazers_count = repo.get("stargazers_count", 0)

        if language and stargazers_count.isdigit():  # Ignore missing languages and non-digit star counts
            language_stars[language] += int(stargazers_count)
            language_counts[language] += 1

# Step 2: Calculate average stars per repository for each language
language_averages = {}
for language in language_stars:
    count = language_counts[language]
    if count > 0:
        average_stars = language_stars[language] / count
        language_averages[language] = average_stars

# Step 3: Find the language with the highest average stars per repository
if language_averages:
    highest_avg_language = max(language_averages, key=language_averages.get)
    highest_avg_stars = language_averages[highest_avg_language]

    print(f"The language with the highest average number of stars per repository is: "
          f"{highest_avg_language} with an average of {highest_avg_stars:.2f} stars.")
else:
    print("No data available to determine the average stars per repository.")


In [ ]:
#Top 5 users by leader strengths.
import csv

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize a list to store user data with leader strength
leader_strengths = []

# Step 1: Read users from users.csv and calculate leader_strength
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        login = user.get("login")
        followers = int(user.get("followers", 0))
        following = int(user.get("following", 0))

        # Calculate leader_strength
        leader_strength = followers / (1 + following) if following >= 0 else followers

        # Append login and leader strength to the list
        leader_strengths.append((login, leader_strength))

# Step 2: Sort users by leader_strength in descending order
top_users = sorted(leader_strengths, key=lambda x: x[1], reverse=True)[:5]

# Step 3: Get the logins of the top 5 users
top_logins = [user[0] for user in top_users]

# Print the result as a comma-separated string
print("Top 5 users by leader strength (logins):", ", ".join(top_logins))


In [ ]:
#Correlation between Number of followers and Number of public repositories.
import csv
from scipy.stats import pearsonr

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize lists to store followers and public repository counts
followers = []
public_repos = []

# Step 1: Read users from users.csv and extract followers and public_repos
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        followers_count = int(user.get("followers", 0))
        public_repos_count = int(user.get("public_repos", 0))

        followers.append(followers_count)
        public_repos.append(public_repos_count)

# Step 2: Calculate the correlation between followers and public repositories
if len(followers) > 1 and len(public_repos) > 1:  # Ensure there is enough data
    correlation, _ = pearsonr(followers, public_repos)
    print(f"The correlation between the number of followers and the number of public repositories is: {correlation:.4f}")
else:
    print("Not enough data to calculate correlation.")


In [ ]:
#Regression Slope of the followers and number of repositories.
import csv
import numpy as np
import statsmodels.api as sm

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize lists to store followers and public repository counts
followers = []
public_repos = []

# Step 1: Read users from users.csv and extract followers and public_repos
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        followers_count = int(user.get("followers", 0))
        public_repos_count = int(user.get("public_repos", 0))

        followers.append(followers_count)
        public_repos.append(public_repos_count)

# Step 2: Prepare the data for regression analysis
X = np.array(public_repos)  # Independent variable (public repositories)
y = np.array(followers)      # Dependent variable (followers)

# Add a constant to the independent variable (intercept)
X = sm.add_constant(X)

# Step 3: Fit the linear regression model
model = sm.OLS(y, X).fit()

# Step 4: Get the estimated coefficient for public repositories
additional_followers_per_repo = model.params[1]  # Coefficient for public_repos
print(f"Estimated additional followers per additional public repository: {additional_followers_per_repo:.4f}")

# Optional: Print the model summary for more insights
print(model.summary())


In [ ]:
import csv

# File paths for repositories.csv
input_csv_path = "/content/repositories.csv"
output_csv_path = "/content/repositories.csv"

# Initialize a list to store processed repository data
processed_repos = []

# Step 1: Read and process repositories.csv
with open(input_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    for repo in repos_reader:
        # Convert empty or None values in has_project and has_wiki to "Null"
        repo["has_projects"] = repo["has_projects"] if repo["has_projects"] else "Null"
        repo["has_wiki"] = repo["has_wiki"] if repo["has_wiki"] else "Null"

        # Append the processed repo data to the list
        processed_repos.append(repo)

# Step 2: Write the processed data back to a new CSV file
with open(output_csv_path, mode="w", newline='', encoding="utf-8") as output_file:
    # Use the fieldnames from the original file to maintain the same structure
    fieldnames = processed_repos[0].keys()
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)

    # Write the header and data
    writer.writeheader()
    writer.writerows(processed_repos)

print(f"Processed data with 'Null' values saved to {output_csv_path}")


In [ ]:
#Correlation between number of followers and number of repositories.
import csv
from scipy.stats import pearsonr

# File path for repositories.csv
repos_csv_path = "/content/repositories.csv"

# Initialize lists to store has_projects and has_wiki flags
has_projects = []
has_wiki = []

# Step 1: Read repositories from repositories.csv and extract has_projects and has_wiki
with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)
    count = 0
    for repo in repos_reader:
        count+=1
        # Convert the string to a boolean
        projects_enabled = repo.get("has_projects") == 'True' # True if the string is "True"
        # print(type(repo.get("has_projects")))
        # print(projects_enabled)
        wiki_enabled = repo.get("has_wiki") == 'True'  # True if the string is "True"

        # projects_enabled_n = repo.get("has_projects") == 'False'  # True if the string is "True"
        # wiki_enabled_n = repo.get("has_wiki") == 'False'  # True if the string is "True"

        has_projects.append(1 if projects_enabled else 0)  # 1 for True, 0 for False
        has_wiki.append(1 if wiki_enabled else 0)  # 1 for True, 0 for False
    print(count)
    print(has_projects.count(1))
    print(has_wiki.count(1))
    print(len(has_wiki))
# Step 2: Calculate the correlation between has_projects and has_wiki
if len(has_projects) > 1 and len(has_wiki) > 1:  # Ensure there is enough data
    correlation,_ = pearsonr(has_projects, has_wiki)
    print(f"The correlation between enabling projects and having wikis enabled is: {correlation:.4f}")
else:
    print("Not enough data to calculate correlation.")


In [ ]:
#Difference in Average hireable and Non-hireable.
import csv

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize lists to store following counts for hireable and non-hireable users
hireable_following = []
non_hireable_following = []

# Step 1: Read users from users.csv and separate based on hireable status
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        following_count = int(user.get("following", 0))
        hireable_status = user.get("hireable") == 'true' # Convert to boolean

        if hireable_status:
            hireable_following.append(following_count)
        else:
            non_hireable_following.append(following_count)

# Step 2: Calculate the average number of people followed for each group
average_hireable = (sum(hireable_following) / len(hireable_following)) if hireable_following else 0
average_non_hireable = (sum(non_hireable_following) / len(non_hireable_following)) if non_hireable_following else 0

# Step 3: Calculate the difference between the averages
difference = average_hireable - average_non_hireable

# Step 4: Print the results
print(f"Average following for hireable users: {average_hireable:.3f}")
print(f"Average following for non-hireable users: {average_non_hireable:.3f}")
print(f"Difference in average following (hireable - non-hireable): {difference:.3f}")


In [ ]:
#Finding most common surname in the data.
import csv
from collections import Counter

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize a list to store surnames
surnames = []

# Step 1: Read users from users.csv and extract the last word in each name as the surname
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        name = user.get("name")

        # Check if name exists and is not empty
        if name:
            # Split the name by whitespace and take the last word as the surname
            surname = name.strip().split()[-1]
            surnames.append(surname)

# Step 2: Count occurrences of each surname
surname_counts = Counter(surnames)

# Step 3: Find the most common surname(s)
if surname_counts:
    max_count = max(surname_counts.values())
    most_common_surnames = sorted([surname for surname, count in surname_counts.items() if count == max_count])
    print(f"The most common surname(s): {', '.join(most_common_surnames)}")
else:
    print("No surnames found in the data.")


In [ ]:
#Regression slope on word count of bio
import csv
from sklearn.linear_model import LinearRegression
import numpy as np

# File path for users.csv
users_csv_path = "/content/users.csv"

# Initialize lists to store bio lengths and follower counts
bio_lengths = []
followers = []

# Step 1: Read users.csv, filter users with bios, and calculate bio length
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        bio = user.get("bio")
        follower_count = int(user.get("followers", 0))

        # Ignore users without a bio
        if bio:
            # Count the number of words in the bio (split by whitespace)
            bio_length = len(bio.split())
            bio_lengths.append(bio_length)
            followers.append(follower_count)

# Step 2: Perform Linear Regression
# Convert lists to NumPy arrays for regression analysis
bio_lengths = np.array(bio_lengths).reshape(-1, 1)
followers = np.array(followers)

# Initialize and fit the regression model
reg_model = LinearRegression()
reg_model.fit(bio_lengths, followers)

# Retrieve the slope (impact per word)
slope = reg_model.coef_[0]

# Output the slope rounded to 3 decimal places
print(f"Regression slope of followers on bio word count: {slope:.3f}")


In [ ]:
#Top 5 users who created repos on weekend.
import csv
from datetime import datetime

# File path for repositories.csv
repos_csv_path = "/content/repositories.csv"

# Dictionary to count weekend repositories for each user
weekend_repos_count = {}

# Step 1: Read repositories.csv and count weekend repositories per user
with open(repos_csv_path, mode="r", encoding="utf-8") as repos_file:
    repos_reader = csv.DictReader(repos_file)

    for repo in repos_reader:
        login = repo.get("login")
        created_at = repo.get("created_at")

        # Parse the created_at date
        created_date = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")

        # Check if the day is a weekend (Saturday=5, Sunday=6)
        if created_date.weekday() >= 5:
            if login in weekend_repos_count:
                weekend_repos_count[login] += 1
            else:
                weekend_repos_count[login] = 1

# Step 2: Identify the top 5 users with the most weekend repositories
top_5_users = sorted(weekend_repos_count, key=weekend_repos_count.get, reverse=True)[:5]

# Output the top 5 users, comma-separated
print("Top 5 users who created the most repositories on weekends (UTC):", ", ".join(top_5_users))



In [ ]:
#Difference in the fraction of hirable and nonhireable (hireable people who share their emails.)
import csv

# File path for users.csv
users_csv_path = "/content/users.csv"

# Counters for hireable and non-hireable users with and without emails
hireable_with_email = 0
hireable_total = 0
non_hireable_with_email = 0
non_hireable_total = 0

# Step 1: Read users.csv and calculate counts
with open(users_csv_path, mode="r", encoding="utf-8") as users_file:
    users_reader = csv.DictReader(users_file)

    for user in users_reader:
        email = user.get("email")
        hireable = user.get("hireable").lower() == "true"  # Convert to boolean

        if hireable:
            hireable_total += 1
            if email:
                hireable_with_email += 1
        else:
            non_hireable_total += 1
            if email:
                non_hireable_with_email += 1

# Step 2: Calculate fractions and difference
hireable_fraction = hireable_with_email / hireable_total if hireable_total > 0 else 0
non_hireable_fraction = non_hireable_with_email / non_hireable_total if non_hireable_total > 0 else 0
fraction_difference = hireable_fraction - non_hireable_fraction

# Output the difference rounded to 3 decimal places
print(f"Difference in fraction: {fraction_difference:.3f}")
